In [105]:
%pip install -q -U langchain langchain-google-genai google_generativeai

In [106]:
%pip show langchain

Name: langchain
Version: 0.3.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [107]:
from google.colab import userdata
import google.generativeai as genai
from google.generativeai import GenerativeModel

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

llm: GenerativeModel = GenerativeModel("gemini-1.5-flash")

In [108]:
def get_completion(prompt: str, model=llm) -> str:
  response = model.generate_content(prompt)
  return response.text

get_completion("What is 9 + 1?")

'9 + 1 = 10 \n'

In [109]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [110]:
style = """American English \
in a calm and respectful tone
"""

In [111]:
prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [112]:
res = get_completion(prompt)
res

"I'm quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I would appreciate any advice you can offer. \n"

Gemini using LangChain\
Now doing the same using LangChain.

In [113]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI =  ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0, api_key=GEMINI_API_KEY)

In [114]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f0916ca0c10>, default_metadata=())

In [115]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [116]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

# print(prompt_template)

# print(prompt_template.messages[0].prompt)

prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [117]:
customer_style = """American English \
in a calm and respectful tone
"""

In [118]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [119]:
customer_message = prompt_template.format_messages(style=customer_style, text=customer_email)

# customer_message
customer_message[0]

HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, response_metadata={})

In [120]:
print(type(customer_message))
print(type(customer_message[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [123]:
res = llm.invoke(customer_message)

# res
res.content

"I'm quite frustrated that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up the mess. I could really use your help right now. \n"